<a href="https://colab.research.google.com/github/pajannat/my-learning/blob/main/KIKAGAKU/pl_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pytorch_lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.8/825.8 KB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.4/512.4 KB 53.1 MB/s eta 0:00:00


In [2]:
import pytorch_lightning as pl

In [3]:
pl.__version__

'1.9.0'

In [59]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchmetrics.functional import accuracy

### データの準備

In [5]:
# データの準備
from sklearn.datasets import load_iris
iris = load_iris()
x = iris['data']
t = iris['target']

In [6]:
# Pytorch の Tensor 型へ変換
x = torch.tensor(x, dtype=torch.float32)
t = torch.tensor(t, dtype=torch.int64)

In [7]:
# 入力値と目標値をまとめる
dataset = torch.utils.data.TensorDataset(x, t)

In [8]:
# 各データセットのサンプル数を決定
# train : val : test = 60% : 20% : 20%
n_train = int(len(dataset) * 0.6)
n_val = int(len(dataset) * 0.2)
n_test = len(dataset) - n_train - n_val

In [9]:
# データセットの分割
# ランダムに分割を行うため、シードを固定して再現性を確保
torch.manual_seed(0)
train, val, test = torch.utils.data.random_split(dataset, [n_train, n_val, n_test])

In [10]:
# バッチサイズの定義
batch_size = 10
# DataLoader を準備
# suffle はデフォルトで False のため、訓練データのみ True に指定
train_loader = torch.utils.data.DataLoader(train, batch_size, shuffle=True, drop_last=True)
val_loader = torch.utils.data.DataLoader(val, batch_size)
test_loader = torch.utils.data.DataLoader(test, batch_size)

### ネットワークの定義

In [11]:
# 4 -> 4-> 3

In [53]:
class Net(pl.LightningModule):

  def __init__(self):
    super().__init__()

    self.fc1 = nn.Linear(4, 4)
    self.fc2 = nn.Linear(4, 3)

  def forward(self, x):
    h = self.fc1(x)
    h = F.relu(h)
    h = self.fc2(h)
    return h

  # 学習データに対する処理
  def training_step(self, batch, batch_idx):
    x, t = batch
    y = self(x)
    loss = F.cross_entropy(y, t)
    # ログを追加
    self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
    self.log('train_acc', 
             accuracy(y, t, task='multiclass', num_classes=len(y[0])),
             on_step=True, on_epoch=True, prog_bar=True)    
    return loss

  # 検証データに対する処理
  def validation_step(self, batch, batch_idx):
    x, t = batch
    y = self(x)
    loss = F.cross_entropy(y, t)
    # ログを追加
    self.log('val_loss', loss, on_step=True, on_epoch=True)
    self.log('val_acc', 
             accuracy(y, t, task='multiclass', num_classes=len(y[0])),
             on_step=True, on_epoch=True)    
    return loss

  # テストデータに対する処理
  def test_step(self, batch, batch_idx):
    x, t = batch
    y = self(x)
    loss = F.cross_entropy(y, t)
    # ログを追加
    self.log('test_loss', loss, on_step=True, on_epoch=True)
    self.log('test_acc', 
             accuracy(y, t, task='multiclass', num_classes=len(y[0])),
             on_step=True, on_epoch=True)    
    return loss

  def configure_optimizers(self):
    optimizer = torch.optim.SGD(self.parameters(), lr=0.01)
    return optimizer

In [54]:
# GPU含めすべてのseedを固定
pl.seed_everything(0)

net = Net()

INFO:lightning_fabric.utilities.seed:Global seed set to 0


In [55]:
net

Net(
  (fc1): Linear(in_features=4, out_features=4, bias=True)
  (fc2): Linear(in_features=4, out_features=3, bias=True)
)

In [56]:
# 学習を行う Trainer
trainer = pl.Trainer(max_epochs=30, gpus=1, deterministic=True)

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:467: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [57]:
# 学習の実行
trainer.fit(net, train_loader, val_loader)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name | Type   | Params
--------------------------------
0 | fc1  | Linear | 20    
1 | fc2  | Linear | 15    
--------------------------------
35        Trainable params
0         Non-trainable params
35        Total params
0.000     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/trainer.py:1600: PossibleUserWarning: The number of training batches (9) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=30` reached.


### 評価指標

In [58]:
trainer.callback_metrics

{'train_loss': tensor(0.7549),
 'train_loss_step': tensor(0.9137),
 'train_acc': tensor(0.8556),
 'train_acc_step': tensor(0.8000),
 'val_loss': tensor(0.6431),
 'val_loss_epoch': tensor(0.6431),
 'val_acc': tensor(0.8667),
 'val_acc_epoch': tensor(0.8667),
 'train_loss_epoch': tensor(0.7549),
 'train_acc_epoch': tensor(0.8556)}

In [62]:
results = trainer.test(dataloaders=test_loader)

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:124: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
  rank_zero_warn(
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/lightning_logs/version_4/checkpoints/epoch=29-step=270.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at /content/lightning_logs/version_4/checkpoints/epoch=29-step=270.ckpt


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     test_acc_epoch         0.9333333373069763
     test_loss_epoch        0.7351360321044922
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [63]:
results

[{'test_loss_epoch': 0.7351360321044922, 'test_acc_epoch': 0.9333333373069763}]